In [1]:
import tensorflow as tf
import pickle
import sys
from scipy.spatial.distance import mahalanobis

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X = pickle.load(open('../Data/X_train_triplet', 'rb'))
test = pickle.load(open('../Data/X_test_triplet', 'rb'))
y = pickle.load(open('../Data/y_test', 'rb'))

cov = pickle.load(open('../Data/covariance','rb'))
cov.shape

(978, 978)

In [3]:
class siamese:

    # Create model
    def __init__(self):
        tf.reset_default_graph()
        self.x1 = tf.placeholder(tf.float32, [None, 978])
        self.x2 = tf.placeholder(tf.float32, [None, 978])
        self.x3 = tf.placeholder(tf.float32, [None, 978])
        self.cov = tf.placeholder(tf.float32, [978,978])
        with tf.variable_scope("siamese") as scope:
            self.o1 = self.network(self.x1)
            scope.reuse_variables()
            self.o2 = self.network(self.x2)
            scope.reuse_variables()
            self.o3 = self.network(self.x3)

        self.loss = self.loss_with_cosine()

    def network(self, x):
        weights = []
        fc1 = self.fc_layer(x, 1024, "fc1")
        ac1 = tf.nn.relu(fc1)
        fc2 = self.fc_layer(ac1, 1024, "fc2")
        ac2 = tf.nn.relu(fc2)
        fc3 = self.fc_layer(ac2, 32, "fc3")
        fc3 = tf.nn.l2_normalize(fc3, axis=1)
        return fc3

    def fc_layer(self, bottom, n_weight, name):
        assert len(bottom.get_shape()) == 2
        n_prev_weight = bottom.get_shape()[1]
        initer = tf.truncated_normal_initializer(stddev=0.01)
        W = tf.get_variable(name+'W', dtype=tf.float32, shape=[n_prev_weight, n_weight], initializer=initer)
        b = tf.get_variable(name+'b', dtype=tf.float32, initializer=tf.constant(0.01, shape=[n_weight], dtype=tf.float32))
        fc = tf.nn.bias_add(tf.matmul(bottom, W), b)
        return fc

    def loss_with_euclid(self):  
        eucd2 = tf.pow(tf.subtract(self.o1, self.o2), 2)
        eucd2 = tf.sqrt(tf.reduce_sum(eucd2, axis=1))
        eucd2_mean = tf.reduce_mean(eucd2, axis=0)
        
        eucd3 = tf.pow(tf.subtract(self.o1, self.o3), 2)
        eucd3 = tf.sqrt(tf.reduce_sum(eucd3, axis=1))
        eucd3_mean = tf.reduce_mean(eucd3, axis=0)
        
        return eucd2_mean, eucd3_mean, eucd2, eucd3
    
    def loss_with_cosine(self):  
        eucd2 = tf.multiply(self.o1, self.o2)
        eucd2 = 1-tf.pow(tf.reduce_sum(eucd2, axis=1), 2)
        eucd2_mean = tf.reduce_mean(eucd2, axis=0)
        
        eucd3 = tf.multiply(self.o1, self.o3)
        eucd3 = 1-tf.pow(tf.reduce_sum(eucd3, axis=1), 2)
        eucd3_mean = tf.reduce_mean(eucd3, axis=0)
        
        return eucd2_mean, eucd3_mean, eucd2, eucd3
    
    def loss_with_mahalanobis(self):
        m1 = mahalanobis(self.o1, self.o2, self.cov)
        m2 = mahalanobis(self.o1, self.o3, self.cov)
        return m1, m2

In [4]:
s = siamese()
optim_pos = tf.train.AdamOptimizer(0.005).minimize(s.loss[0])
optim_neg = tf.train.AdamOptimizer(0.005).minimize(-s.loss[1])

In [ ]:
with tf.Session() as session:  
    tf.initialize_all_variables().run()
    print("Initialized")
    
    feed_dict={s.x1:X[0], s.x2:X[1], s.x3:X[2]}
    
    p_loss = []
    n_loss = []
    
    for a in range(40):        
        p_loss.append([])
        n_loss.append([])
        for b in range(len(X[0])):
            sys.stdout.write('\rEpoch %d:\t%d/%d' % (a, b+1, len(X[0])))
            feed_dict = {s.x1:X[0][b], s.x2:X[1][b], s.x3:X[2][b], s.cov=cov}
            _, _, l = session.run([optim_pos, optim_neg, s.loss], feed_dict=feed_dict)
            p_loss[-1].append(l[0])
            n_loss[-1].append(l[1])
        p_loss[-1] = sum(p_loss[-1])/len(p_loss[-1])
        n_loss[-1] = sum(n_loss[-1])/len(n_loss[-1])
        print(" ", p_loss[-1], n_loss[-1]) 
    
    embeddings = session.run([s.o1, s.o2, s.o3], feed_dict={s.x1:X[0], s.x2:X[1], s.x3:X[2],s.cov=cov})
    trained = session.run([s.loss], feed_dict={s.x1:X[0], s.x2:X[1], s.x3:X[2],s.cov=cov})
    pred = session.run([s.loss], feed_dict={s.x1:test[0], s.x2:test[1], s.x3:test[2],s.cov=cov})
    
    

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(np.arange(40), p_loss, label='positive')
plt.plot(np.arange(40), n_loss, label='negative')
plt.legend()
plt.show()

In [ ]:
values = []
for i in pred:
    anch = normalize(i[0:16].reshape(1,-1)).flatten()
    pos = normalize(i[16:32].reshape(1,-1)).flatten()
    neg = normalize(i[32:].reshape(1,-1)).flatten()
    pos_neg_val = [(np.dot(anch,pos)), (np.dot(anch,neg))]
    print(pos_neg_val)
    values.append(pos_neg_val[0]>=0.7)
    values.append(pos_neg_val[1]<0.7)
    # print(positives, negatives)
    #     print(pos_val, neg_val, pos_val>neg_val)
print((np.sum(np.asarray(values)==True))/len(values))

In [ ]:
p = np.sum(trained[0][2]<=0.5)
n = np.sum(trained[0][3]>0.5)
print(p, n, (p+n)/420/2)

In [ ]:
p = np.sum(pred[0][2]<=0.5)
n = np.sum(pred[0][3]>0.5)
print(p, n, (p+n)/420/2)

In [ ]:
x = 138
print(trained[0][2][x], trained[0][3][x])
print(embeddings[0][x], embeddings[1][x], embeddings[2][x], sep='\n') 

In [5]:
from gensim.models.keyedvectors import KeyedVectors

In [6]:
word2vec = KeyedVectors.load_word2vec_format('./doc2vec_model.doc2vec', binary=True, unicode_errors='ignore')

FileNotFoundError: [Errno 2] No such file or directory: './doc2vec_model.doc2vec'